In [11]:
import requests, json
from pprint import pprint
import pandas as pd
from tqdm.auto import tqdm

c:\Users\fitzm\anaconda3\envs\fantasy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Functions

In [12]:

def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df

def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


In [13]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'chips': [...],
  'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_config': {...},
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11095555}


In [14]:
pd.set_option('display.max_columns', None)
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Fábio Vieira,1,3
1,2,G.Jesus,1,4
2,3,Gabriel,1,2
3,4,Havertz,1,4
4,5,Hein,1,1


In [15]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

In [16]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],73
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],239
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],319
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],78


In [17]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Fábio,Ferreira Vieira,Arsenal
1,Gabriel,Fernando de Jesus,Arsenal
2,Gabriel,dos Santos Magalhães,Arsenal
3,Kai,Havertz,Arsenal
4,Karl,Hein,Arsenal


In [18]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Fábio,Ferreira Vieira,Arsenal,Midfielder
1,Gabriel,Fernando de Jesus,Arsenal,Forward
2,Gabriel,dos Santos Magalhães,Arsenal,Defender
3,Kai,Havertz,Arsenal,Forward
4,Karl,Hein,Arsenal,Goalkeeper


Player Gameweek History

In [19]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


Points table of gameweek points for all weeks

In [20]:
tqdm.pandas()

# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()


,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,MID
1,2,Gabriel,Fernando de Jesus,G.Jesus,Arsenal,FWD
2,3,Gabriel,dos Santos Magalhães,Gabriel,Arsenal,DEF
3,4,Kai,Havertz,Havertz,Arsenal,FWD
4,5,Karl,Hein,Hein,Arsenal,GKP


In [21]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

100%|██████████| 709/709 [00:34<00:00, 20.41it/s]


In [22]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head()

,element,web_name,total_points,goals_scored,assists
327,328,M.Salah,202,17,13
181,182,Palmer,143,12,7
98,99,Mbeumo,117,11,2
400,401,Isak,112,12,4
540,541,Cunha,111,10,4
